# Exercise 12: Introduction to PyROOT

In [1]:
import math
import numpy as np
import pandas as pd
import ROOT

In [2]:
# --- 1. Load CSV (unlabeled data) ---
df = pd.read_csv("data/sample_ex11.csv")
x = df["x"].to_numpy()
print("Loaded:", df.shape, "rows")

Loaded: (52000, 1) rows


In [4]:
# --- 2. Define histogram (TH1F) and fill ---
nbins = 80
h = ROOT.TH1F("h_x", "x distribution; x; counts", nbins, 0, 40)
for val in x:
    h.Fill(val)

c1 = ROOT.TCanvas("c1", "c1", 800, 600)
h.Draw()
c1.Update()


Warning in <TROOT::Append>: Replacing existing TH1: h_x (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [16]:
# --- 3. Define model TF1: exp + Gaussian (unnormalized Gaussian) ---
# f(x) = [0]*exp(-x/[1]) + [2]*exp(-0.5*((x-[3])/[4])^2)
# Parameters: [0]=B, [1]=tau, [2]=A, [3]=mu, [4]=sigma

func = ROOT.TF1("f_exp_gaus",
               "[0]*exp(-x/[1]) + [2]*exp(-0.5*((x-[3])/[4])**2)",
               0, 40)

# --- Initial seeds ---
B = max(h.GetBinContent(1), 1.0)
tau = 3.0
A = h.GetMaximum()
mu  = h.GetBinCenter(h.GetMaximumBin())
sigma = max(0.5, h.GetRMS()/2.0)

func.SetParameters(B, tau, A, mu, sigma)
# Optional reasonable limits
func.SetParLimits(0, 0, 1e5)    # B
func.SetParLimits(1, 0.05, 50)  # tau
func.SetParLimits(2, 0, 1e3)    # A
func.SetParLimits(3, 6, 10)     # mu
func.SetParLimits(4, 0.05, 5.0) # sigma


In [20]:
# --- 4. Fit histogram (chi2 fit with Poisson-like errors) ---
h.Fit(func, "R")

c2 = ROOT.TCanvas("c2", "c2", 800, 600)
h.Draw()
func.Draw("same")
c2.Update()

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      33.6384
NDf                       =           53
Edm                       =  5.34707e-07
NCalls                    =           64
p0                        =      8396.75   +/-   53.8363      	 (limited)
p1                        =      2.97524   +/-   0.0149174    	 (limited)
p2                        =      777.793   +/-   31.1283      	 (limited)
p3                        =      8.03658   +/-   0.0221841    	 (limited)
p4                        =     0.529496   +/-   0.0212877    	 (limited)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2


In [18]:
# --- 5. Extract parameters and compute yields ---
B     = func.GetParameter(0)
B_err = func.GetParError(0)

tau     = func.GetParameter(1)
tau_err = func.GetParError(1)

A     = func.GetParameter(2)
A_err = func.GetParError(2)

mu     = func.GetParameter(3)
mu_err = func.GetParError(3)

sigma     = func.GetParameter(4)
sigma_err = func.GetParError(4)

nbins = h.GetNbinsX()
centers = np.array([h.GetXaxis().GetBinCenter(i) for i in range(1, nbins+1)])


In [19]:
# Per-bin component predictions at bin centers
bkg_per_bin = B * np.exp(-centers / tau)
sig_per_bin = A * np.exp(-0.5 * ((centers - mu)/sigma)**2)

# Yields (sum of per-bin expectations)
bkg_yield = float(np.sum(bkg_per_bin))
sig_yield = float(np.sum(sig_per_bin))

# Diagnostics
total_obs = h.Integral()
total_fit = bkg_yield + sig_yield

print("Fit results:")
print(f"B = {B:.3g} ± {B_err:.3g}")
print(f"tau = {tau:.3g} ± {tau_err:.3g}")
print(f"A(gauss) = {A:.3g} ± {A_err:.3g}")
print(f"mu = {mu:.3f} ± {mu_err:.3f}")
print(f"sigma = {sigma:.3f} ± {sigma_err:.3f}")
print(f"Estimated yields from bin-sum:")
print(f"  Signal ≈ {sig_yield:.1f}")
print(f"  Bkg    ≈ {bkg_yield:.1f}")
print(f"  Total (fit) ≈ {total_fit:.1f} vs Observed = {total_obs:.0f}")


Fit results:
B = 8.4e+03 ± 53.8
tau = 2.98 ± 0.0149
amp(gauss) = 778 ± 31.1
mu = 8.037 ± 0.022
sigma = 0.529 ± 0.021
Estimated yields from bin-sum:
  Signal ≈ 2064.6
  Bkg    ≈ 49905.7
  Total (fit) ≈ 51970.3 vs Observed = 52000


In [10]:
# --- 6. Save canvases to files (default) ---
c1.SaveAs("ex12_hist.png")
c2.SaveAs("ex12_fit.png")


Info in <TCanvas::Print>: png file ex12_hist.png has been created
Info in <TCanvas::Print>: png file ex12_fit.png has been created
